Toxic Russian Comments Dataset This dataset contains labelled comments from the popular Russian social network ok.ru.

The data was used in a competition where participants had to automatically label each comment with at least one of the four predefined classes. The classes represent different levels of toxicity. The competition was held on the All Cups platform.

Each comment belongs to one of the following classes, with each label complying with the fastText formatting rules:

__label__NORMAL - neutral user comments

__label__INSULT - comments that humiliate a person

__label__THREAT - comments with an explicit intent to harm another person

__label__OBSCENITY - comments that contain a description or a threat of a sexual assault

Data overview:

count_of_elements: 248290

count_of_labels: 4

label_count:

 __label__NORMAL: 203685

 __label__INSULT: 28567

 __label__INSULT,__label__THREAT: 6317

 __label__THREAT: 5460

 __label__OBSCENITY: 2245

 __label__INSULT,__label__OBSCENITY: 1766

 __label__INSULT,__label__OBSCENITY,__label__THREAT: 176

 __label__OBSCENITY,__label__THREAT: 74

In [4]:
import pandas as pd

In [5]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("alexandersemiletov/toxic-russian-comments")

print("Path to dataset files:", path)

D:\СПбПУ\1_семестр\Методы_обработки_экспериментальных_данных\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████| 12.0M/12.0M [00:00<00:00, 12.7MB/s]

Extracting files...


Path to dataset files: C:\Users\Александра\.cache\kagglehub\datasets\alexandersemiletov\toxic-russian-comments\versions\1


In [8]:
data_list = []
with open("C:/Users/Александра/.cache/kagglehub/datasets/alexandersemiletov/toxic-russian-comments/versions/1/dataset.txt", encoding='utf-8') as file:
    for line in file:
        labels = line.split()[0]
        text = line[len(labels)+1:].strip()
        labels = labels.split(",")
        mask = [1 if "__label__NORMAL" in labels else 0,
                1 if "__label__INSULT" in labels else 0,
                1 if "__label__THREAT" in labels else 0,
                1 if "__label__OBSCENITY" in labels else 0]
        data_list.append((text, *mask))

In [9]:
df = pd.DataFrame(data_list, columns=["text", "normal", "insult", "threat", "obscenity"])

In [10]:
df

,text,normal,insult,threat,obscenity
0,скотина! что сказать,0,1,0,0
1,я сегодня проезжала по рабочей и между домами ...,1,0,0,0
2,очередной лохотрон. зачем придумывать очередно...,1,0,0,0
3,"ретро дежавю ... сложно понять чужое сердце , ...",1,0,0,0
4,а когда мы статус агрогородка получили?,1,0,0,0
...,...,...,...,...,...
248285,правильно всё по пять (5)...,1,0,0,0
248286,ёбанные нубы заходите на сервер мой ник _creep...,0,1,0,0
248287,а у меня наверное рекорд в 1962 году в училище...,1,0,0,0
248288,спасибо всем большое),1,0,0,0


# ФУНКЦИИ ДЛЯ ОЧИСТКИ

In [11]:
import pymorphy2
import re

morph = pymorphy2.MorphAnalyzer()

def clean_text_pymorphy(text):
    text = re.sub(r'[^а-яА-ЯёЁ\s]', '', text)
    text = text.lower()
    tokens = [morph.parse(word)[0].normal_form for word in text.split()]
    return ' '.join(tokens)

text_example = "Это пример текста для очистки с использованием pymorphy2!"
clean_text_pymorphy(text_example)


'это пример текст для очистка с использование'

In [12]:

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('punkt')

stop_words = set(stopwords.words("russian"))

def clean_text_nltk(text):
    text = re.sub(r'[^а-яА-ЯёЁ\s]', '', text.lower())
    words = word_tokenize(text)
    words = [word for word in words if word not in stop_words]
    return ' '.join(words)

# Пример очистки
text_example = "Это пример текста для очистки с использованием nltk"
clean_text_nltk(text_example)


[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Александра\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Александра\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Александра\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


'это пример текста очистки использованием'

In [13]:
from natasha import Doc, Segmenter, MorphVocab, NewsEmbedding, NewsMorphTagger
import re

segmenter = Segmenter()
morph_vocab = MorphVocab()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)

def clean_text_natasha(text):
    text = re.sub(r'[^а-яА-ЯёЁ\s]', '', text.lower())

    doc = Doc(text)
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)

    lemmas = []
    for token in doc.tokens:
        token.lemmatize(morph_vocab)
        lemmas.append(token.lemma)
    return ' '.join(lemmas)

# Пример очистки
text_example = "Это пример текста для очистки с использованием Natasha!"
clean_text_natasha(text_example)

'это пример текст для очистка с использование'

In [14]:
import spacy
import re

nlp = spacy.load("ru_core_news_sm")

def clean_text_spacy(text):
    text = re.sub(r'[^а-яА-ЯёЁ\s]', '', text.lower())
    doc = nlp(text)
    lemmas = [token.lemma_ for token in doc if not token.is_stop]
    return ' '.join(lemmas)
text_example = "Это пример текста для очистки с использованием spacy!"
clean_text_spacy(text_example)

'пример текст очистка использование'

In [22]:
import re

def clean_text_re(text):
    text = re.sub(r'[^а-яА-ЯёЁ\s]', '', text.lower())
    return text
text_example = "Это пример текста для очистки с использованием re!"
clean_text_re(text_example)

'это пример текста для очистки с использованием '

# РЕДАКТИРОВАНИЕ ИСХОДНЫХ ДАТАФРЕЙМОВ

In [15]:
df_no_change = df.copy()
df_pymorphy = df.copy()
df_nltk = df.copy()
df_natasha=df.copy()
df_spacy=df.copy()
df_re=df.copy()

In [16]:
import time

In [17]:
start_time = time.time()
df_pymorphy['cleaned_text'] = df_pymorphy['text'].apply(clean_text_pymorphy)
end = time.time()
print(end-start_time)

531.4316663742065


In [18]:
start_time = time.time()
df_nltk['cleaned_text'] = df_nltk['text'].apply(clean_text_nltk)
end = time.time()
print(end-start_time)

19.920001983642578


In [19]:
start_time = time.time()
df_natasha['cleaned_text'] = df_natasha['text'].apply(clean_text_natasha)
end = time.time()
print(end-start_time)

605.1016881465912


In [20]:
start_time = time.time()
df_spacy['cleaned_text'] = df_spacy['text'].apply(clean_text_spacy)
end = time.time()
print(end-start_time)

2133.9006662368774


In [23]:
start_time = time.time()
df_re['cleaned_text'] = df_re['text'].apply(clean_text_re)
end = time.time()
print(end-start_time)

1.0473580360412598


In [24]:
df_no_change

,text,normal,insult,threat,obscenity
0,скотина! что сказать,0,1,0,0
1,я сегодня проезжала по рабочей и между домами ...,1,0,0,0
2,очередной лохотрон. зачем придумывать очередно...,1,0,0,0
3,"ретро дежавю ... сложно понять чужое сердце , ...",1,0,0,0
4,а когда мы статус агрогородка получили?,1,0,0,0
...,...,...,...,...,...
248285,правильно всё по пять (5)...,1,0,0,0
248286,ёбанные нубы заходите на сервер мой ник _creep...,0,1,0,0
248287,а у меня наверное рекорд в 1962 году в училище...,1,0,0,0
248288,спасибо всем большое),1,0,0,0


In [25]:
df_re

,text,normal,insult,threat,obscenity,cleaned_text
0,скотина! что сказать,0,1,0,0,скотина что сказать
1,я сегодня проезжала по рабочей и между домами ...,1,0,0,0,я сегодня проезжала по рабочей и между домами ...
2,очередной лохотрон. зачем придумывать очередно...,1,0,0,0,очередной лохотрон зачем придумывать очередной...
3,"ретро дежавю ... сложно понять чужое сердце , ...",1,0,0,0,ретро дежавю сложно понять чужое сердце лиш ...
4,а когда мы статус агрогородка получили?,1,0,0,0,а когда мы статус агрогородка получили
...,...,...,...,...,...,...
248285,правильно всё по пять (5)...,1,0,0,0,правильно всё по пять
248286,ёбанные нубы заходите на сервер мой ник _creep...,0,1,0,0,ёбанные нубы заходите на сервер мой ник для п...
248287,а у меня наверное рекорд в 1962 году в училище...,1,0,0,0,а у меня наверное рекорд в году в училище кор...
248288,спасибо всем большое),1,0,0,0,спасибо всем большое


In [26]:
df_pymorphy

,text,normal,insult,threat,obscenity,cleaned_text
0,скотина! что сказать,0,1,0,0,скотина что сказать
1,я сегодня проезжала по рабочей и между домами ...,1,0,0,0,я сегодня проезжать по рабочий и между дом сни...
2,очередной лохотрон. зачем придумывать очередно...,1,0,0,0,очередной лохотрон зачем придумывать очередной...
3,"ретро дежавю ... сложно понять чужое сердце , ...",1,0,0,0,ретро дежавю сложно понять чужое сердце лиш ощ...
4,а когда мы статус агрогородка получили?,1,0,0,0,а когда мы статус агрогородок получить
...,...,...,...,...,...,...
248285,правильно всё по пять (5)...,1,0,0,0,правильно всё по пять
248286,ёбанные нубы заходите на сервер мой ник _creep...,0,1,0,0,ёбанный нуба заходить на сервер мой ник для пв...
248287,а у меня наверное рекорд в 1962 году в училище...,1,0,0,0,а у я наверное рекорд в год в училище коренной...
248288,спасибо всем большое),1,0,0,0,спасибо весь большой


In [27]:
df_natasha

,text,normal,insult,threat,obscenity,cleaned_text
0,скотина! что сказать,0,1,0,0,скотина что сказать
1,я сегодня проезжала по рабочей и между домами ...,1,0,0,0,я сегодня проезжать по рабочий и между дом сни...
2,очередной лохотрон. зачем придумывать очередно...,1,0,0,0,очередной лохотрон зачем придумывать очередной...
3,"ретро дежавю ... сложно понять чужое сердце , ...",1,0,0,0,ретро дежавю сложный понять чужой сердце лиш о...
4,а когда мы статус агрогородка получили?,1,0,0,0,а когда мы статус агрогородок получить
...,...,...,...,...,...,...
248285,правильно всё по пять (5)...,1,0,0,0,правильно весь по пять
248286,ёбанные нубы заходите на сервер мой ник _creep...,0,1,0,0,ебанный нуба заходить на сервер мой ник для пв...
248287,а у меня наверное рекорд в 1962 году в училище...,1,0,0,0,а у я наверное рекорд в год в училище коренной...
248288,спасибо всем большое),1,0,0,0,спасибо весь большой


In [28]:
df_nltk

,text,normal,insult,threat,obscenity,cleaned_text
0,скотина! что сказать,0,1,0,0,скотина сказать
1,я сегодня проезжала по рабочей и между домами ...,1,0,0,0,сегодня проезжала рабочей домами снитенко гомо...
2,очередной лохотрон. зачем придумывать очередно...,1,0,0,0,очередной лохотрон придумывать очередной налог...
3,"ретро дежавю ... сложно понять чужое сердце , ...",1,0,0,0,ретро дежавю сложно понять чужое сердце лиш ощ...
4,а когда мы статус агрогородка получили?,1,0,0,0,статус агрогородка получили
...,...,...,...,...,...,...
248285,правильно всё по пять (5)...,1,0,0,0,правильно всё пять
248286,ёбанные нубы заходите на сервер мой ник _creep...,0,1,0,0,ёбанные нубы заходите сервер ник пвп переебу п...
248287,а у меня наверное рекорд в 1962 году в училище...,1,0,0,0,наверное рекорд году училище коренной зубвозмо...
248288,спасибо всем большое),1,0,0,0,спасибо всем большое


# УДАЛЯЕМ НЕОЧИЩЕННЫЙ ТЕКСТ ИЗ ДАТАФРЕЙМОВ

In [30]:
df_re = df_re.drop(columns=['text'])
df_pymorphy = df_pymorphy.drop(columns=['text'])
df_natasha = df_natasha.drop(columns=['text'])
df_nltk = df_nltk.drop(columns=['text'])
df_spacy = df_spacy.drop(columns=['text'])

In [71]:
df_no_change = df_no_change.rename(columns={'text': 'cleaned_text'})

# ОБУЧЕНИЕ

In [63]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

****RE****

In [80]:
start_time = time.time()
# Загрузка данных
df = df_re.copy()

# Целевые метки и текст
X = df["cleaned_text"]
y = df[["normal", "insult", "threat", "obscenity"]]

# Преобразование текста
tfidf = TfidfVectorizer(max_features=5000, ngram_range=(1, 2))  # Учитываем униграммы и биграммы
X_tfidf = tfidf.fit_transform(X)

# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

base_rf = RandomForestClassifier(n_estimators=50, random_state=42, n_jobs=-1, class_weight='balanced')

# Обучение модели для многозадачной классификации
model_re = MultiOutputClassifier(base_rf, n_jobs=-1)
model_re.fit(X_train, y_train)

# Предсказание
y_pred = model_re.predict(X_test)

# Оценка качества
print("Отчет по классификации:")
print(classification_report(y_test, y_pred, target_names=y.columns))
print(f"Time:{time.time()-start_time}")


Отчет по классификации:
              precision    recall  f1-score   support

      normal       0.94      0.97      0.95     40669
      insult       0.81      0.65      0.72      7462
      threat       0.77      0.60      0.67      2420
   obscenity       0.39      0.37      0.38       836

   micro avg       0.91      0.89      0.90     51387
   macro avg       0.72      0.65      0.68     51387
weighted avg       0.90      0.89      0.90     51387
 samples avg       0.90      0.90      0.90     51387

Time:601.4649004936218


D:\СПбПУ\1_семестр\Методы_обработки_экспериментальных_данных\.venv\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


****PYMORPHY****

In [79]:
start_time = time.time()
# Загрузка данных
df = df_pymorphy.copy()

# Целевые метки и текст
X = df["cleaned_text"]
y = df[["normal", "insult", "threat", "obscenity"]]

# Преобразование текста
tfidf = TfidfVectorizer(max_features=5000, ngram_range=(1, 2))  # Учитываем униграммы и биграммы
X_tfidf = tfidf.fit_transform(X)

# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)


base_rf = RandomForestClassifier(n_estimators=50, random_state=42, n_jobs=-1, class_weight='balanced')

# Обучение модели для многозадачной классификации
model_py = MultiOutputClassifier(base_rf, n_jobs=-1)
model_py.fit(X_train, y_train)

# Предсказание
y_pred = model_py.predict(X_test)

# Оценка качества
print("Отчет по классификации:")
print(classification_report(y_test, y_pred, target_names=y.columns))
print(f"Time:{time.time()-start_time}")


Отчет по классификации:
              precision    recall  f1-score   support

      normal       0.95      0.97      0.96     40669
      insult       0.85      0.71      0.78      7462
      threat       0.76      0.58      0.66      2420
   obscenity       0.57      0.46      0.51       836

   micro avg       0.92      0.91      0.92     51387
   macro avg       0.78      0.68      0.73     51387
weighted avg       0.92      0.91      0.91     51387
 samples avg       0.92      0.92      0.92     51387

Time:581.2061095237732


D:\СПбПУ\1_семестр\Методы_обработки_экспериментальных_данных\.venv\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


****NLTK****

In [78]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
import time

start_time = time.time()

# Загрузка данных
df = df_nltk.copy()

# Целевые метки и текст
X = df["cleaned_text"]
y = df[["normal", "insult", "threat", "obscenity"]]

# Преобразование текста
tfidf = TfidfVectorizer(max_features=5000, ngram_range=(1, 2))
X_tfidf = tfidf.fit_transform(X)

# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

# Создание модели
base_rf = RandomForestClassifier(n_estimators=50, random_state=42, n_jobs=-1, class_weight='balanced')

# Обучение модели для многозадачной классификации
model_nltk = MultiOutputClassifier(base_rf, n_jobs=-1)
model_nltk.fit(X_train, y_train)

# Предсказание
y_pred = model_nltk.predict(X_test)

# Оценка качества
print("Отчет по классификации:")
print(classification_report(y_test, y_pred, target_names=y.columns))
print(f"Time: {time.time()-start_time}")


Отчет по классификации:
              precision    recall  f1-score   support

      normal       0.94      0.97      0.95     40669
      insult       0.79      0.70      0.74      7462
      threat       0.65      0.68      0.66      2420
   obscenity       0.31      0.44      0.37       836

   micro avg       0.89      0.91      0.90     51387
   macro avg       0.67      0.70      0.68     51387
weighted avg       0.90      0.91      0.90     51387
 samples avg       0.90      0.91      0.91     51387

Time: 720.0705151557922


D:\СПбПУ\1_семестр\Методы_обработки_экспериментальных_данных\.venv\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


****NATASHA****

In [81]:
start_time = time.time()
# Загрузка данных
df = df_natasha.copy()

# Целевые метки и текст
X = df["cleaned_text"]
y = df[["normal", "insult", "threat", "obscenity"]]

# Преобразование текста
tfidf = TfidfVectorizer(max_features=5000, ngram_range=(1, 2))  # Учитываем униграммы и биграммы
X_tfidf = tfidf.fit_transform(X)

# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

base_rf = RandomForestClassifier(n_estimators=50, random_state=42, n_jobs=-1, class_weight='balanced')

# Обучение модели для многозадачной классификации
model_natasha = MultiOutputClassifier(base_rf, n_jobs=-1)
model_natasha.fit(X_train, y_train)

# Предсказание
y_pred = model_natasha.predict(X_test)

# Оценка качества
print("Отчет по классификации:")
print(classification_report(y_test, y_pred, target_names=y.columns))
print(f"Time:{time.time()-start_time}")

Отчет по классификации:
              precision    recall  f1-score   support

      normal       0.94      0.97      0.96     40669
      insult       0.83      0.69      0.76      7462
      threat       0.75      0.57      0.65      2420
   obscenity       0.53      0.45      0.49       836

   micro avg       0.92      0.90      0.91     51387
   macro avg       0.76      0.67      0.71     51387
weighted avg       0.91      0.90      0.91     51387
 samples avg       0.91      0.91      0.91     51387

Time:547.1720204353333


D:\СПбПУ\1_семестр\Методы_обработки_экспериментальных_данных\.venv\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


****SPACY****

In [82]:
start_time = time.time()
# Загрузка данных
df = df_spacy.copy()

# Целевые метки и текст
X = df["cleaned_text"]
y = df[["normal", "insult", "threat", "obscenity"]]

# Преобразование текста
tfidf = TfidfVectorizer(max_features=5000, ngram_range=(1, 2))  # Учитываем униграммы и биграммы
X_tfidf = tfidf.fit_transform(X)

# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

base_rf = RandomForestClassifier(n_estimators=50, random_state=42, n_jobs=-1, class_weight='balanced')

# Обучение модели для многозадачной классификации
model_spacy = MultiOutputClassifier(base_rf, n_jobs=-1)
model_spacy.fit(X_train, y_train)

# Предсказание
y_pred = model_spacy.predict(X_test)

# Оценка качества
print("Отчет по классификации:")
print(classification_report(y_test, y_pred, target_names=y.columns))
print(f"Time:{time.time()-start_time}")

Отчет по классификации:
              precision    recall  f1-score   support

      normal       0.95      0.97      0.96     40669
      insult       0.81      0.74      0.77      7462
      threat       0.64      0.67      0.65      2420
   obscenity       0.10      0.60      0.17       836

   micro avg       0.84      0.92      0.88     51387
   macro avg       0.62      0.75      0.64     51387
weighted avg       0.90      0.92      0.91     51387
 samples avg       0.88      0.92      0.89     51387

Time:627.1555302143097


D:\СПбПУ\1_семестр\Методы_обработки_экспериментальных_данных\.venv\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


****NO CHANGE****

In [84]:
start_time = time.time()
# Загрузка данных
df = df_no_change.copy()

# Целевые метки и текст
X = df["cleaned_text"]
y = df[["normal", "insult", "threat", "obscenity"]]

# Преобразование текста
tfidf = TfidfVectorizer(max_features=5000, ngram_range=(1, 2))  # Учитываем униграммы и биграммы
X_tfidf = tfidf.fit_transform(X)

# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

base_rf1 = RandomForestClassifier(n_estimators=50, random_state=42, n_jobs=-1, class_weight='balanced', )

# Обучение модели для многозадачной классификации
model_no_change = MultiOutputClassifier(base_rf1, n_jobs=-1)
model_no_change.fit(X_train, y_train)

# Предсказание
y_pred = model_no_change.predict(X_test)

# Оценка качества
print("Отчет по классификации:")
print(classification_report(y_test, y_pred, target_names=y.columns))
print(f"Time:{time.time()-start_time}")


Отчет по классификации:
              precision    recall  f1-score   support

      normal       0.95      0.97      0.96     40669
      insult       0.85      0.71      0.78      7462
      threat       0.76      0.58      0.66      2420
   obscenity       0.57      0.46      0.51       836

   micro avg       0.92      0.91      0.92     51387
   macro avg       0.78      0.68      0.73     51387
weighted avg       0.92      0.91      0.91     51387
 samples avg       0.92      0.92      0.92     51387

Time:544.2620096206665


D:\СПбПУ\1_семестр\Методы_обработки_экспериментальных_данных\.venv\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [1]:
import pandas as pd
df = pd.read('C:/Users/Александра/Downloads/ee42af91-bfe1-cf434a948483/Propert_rent_merged.csv')
df

ModuleNotFoundError: No module named 'pandas'